In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install matplotlib
!pip install pylatexenc
!pip install scipy

!git clone https://github.com/jeanfredericlaprade/atelier_qaoa.git

# Optimisation quantique avec QAOA

Dans ce notebook nous allons voir :

1) Comment préparer un **état quantique** à l'aide d'un **circuit quantique** en utilisant la classe `QuantumCircuit` de Qiskit.

2) Comment construire un **observable** en utilisant le sous-module `quantum_info` de Qiskit.

3) Comment estimer le **gain moyen** en mesurant la valeur moyenne d'un **observable** sur un **état quantique** donné.

4) À quoi ressemble un **circuit variationnel** pour **QAOA**.

5) Comment **optimiser** les paramètres d'un circuit variationnel.

6) Comment le **nombre de répétitions** dans le circuit **QAOA** influence les résultats.

## Installons quelques modules qui nous seront utiles

In [1]:
import networkx as nx

from qiskit import QuantumCircuit
from qiskit.circuit.library.n_local.qaoa_ansatz import QAOAAnsatz
from qiskit.primitives import BackendEstimatorV2 as Estimator, BackendSamplerV2 as Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

from scipy.optimize import minimize

In [2]:
import sys
sys.path.insert(0, '/content/atelier_qaoa')

from qaoa_algolab import *

Vous devriez pouvoir éxécuter la cellule suivante si le fichier `QAOA_AlgoLab.py` se trouve dans le même dossier que ce notebook.

In [ ]:
test_function()

## Préparation d'états quantiques

Commençons avec la préparation d'un état quantique en programmant un circuit quantique. Préparons l'état $|01011\rangle$.

In [ ]:
qc_trial = QuantumCircuit(5)
qc_trial.x([0,1,3])
qc_trial.draw('mpl', scale=0.5)

Nous pouvons vérifier que ce circuit prépare bien l'état désiré en simulant son éxécution à l'aide du simulateur `AerSimulator`.

La prochaine cellule ajoute des mesures au circuit `qc_trial`, simule son éxécution et retourne un dictionnaire qui décrit quels résultats on été obtenus et combien de fois chacun (`counts`).

In [ ]:
qc_trial_state = qc_trial.copy()
qc_trial.measure_all()

simulator = AerSimulator(shots=100)  # On répète la préparation et la mesure du circuit 100 fois
counts = simulator.run(qc_trial).result().get_counts()

print(counts)

### Exercice 1

Programmer un circuit quantique qui permet de préparer l'état suivant. 

$ \frac{1}{\sqrt{2}} \big( |11101\rangle + |11001\rangle \big) $

In [ ]:
qc_ex_1 = QuantumCircuit(5)
# Votre code ici

In [ ]:
exercise_superposition_state(qc_ex_1)

## Construire un observable

Utilisons le sous-module `quantum_info` pour construire des **observables**.

La cellule suivante illustre comment construire le premier terme de l'observable de gain associé au lien entre les noeuds rouge et orange dans le graphe des super-héros.

In [ ]:
example_operator = SparsePauliOp(data=["IIIZZ"], coeffs=[-0.5])
print(example_operator)

### Exercice 2

Construisez l'opérateur de gain total pour le graphe complet du problème d'optimisation des super-héroes.
$$
    - \frac{1}{2}\hat{I}\hat{I}\hat{I}\hat{Z}\hat{Z}
    - \frac{1}{2}\hat{I}\hat{I}\hat{Z}\hat{I}\hat{Z}
    - \frac{1}{2}\hat{I}\hat{I}\hat{Z}\hat{Z}\hat{I}
    - \frac{1}{2}\hat{Z}\hat{I}\hat{I}\hat{Z}\hat{I}
    - \frac{1}{2}\hat{I}\hat{Z}\hat{Z}\hat{I}\hat{I} 
    - \frac{1}{2}\hat{Z}\hat{Z}\hat{I}\hat{I}\hat{I} 
    + \frac{6}{2}\hat{I}\hat{I}\hat{I}\hat{I}\hat{I}
$$

In [ ]:
gain_operator = None  # Completer l'observable

exercise_gain_operator(gain_operator)

## Estimer le gain moyen

On combine un `QuantumCircuit` et un **observable** pour **estimer le gain moyen** à l'aide de Qiskit, $  \langle\psi|\hat{G}|\psi\rangle $.

On vous fourni la fonction `eval_observable_on_state()` pour effectuer ce calcul. On reviendra à son implémentation un peu plus tard dans ce notebook.

In [ ]:
data = ["IIIZZ", "IIZIZ", "IIZZI", "ZIIZI", "IZZII", "ZZIII", "IIIII"]
coeffs = [-0.5,  -0.5, -0.5, -0.5, -0.5, -0.5, 0.5*6]

gain_operator = SparsePauliOp(data=data, coeffs=coeffs)

average_gain = eval_observable_on_state(gain_operator, qc_trial_state, simulator)

print(average_gain)

### Exercice 3

Estimer le gain moyen à l'aide de `gain_operator` pour l'état quantique $ \frac{1}{\sqrt{2}} \big( |01011\rangle + |11011\rangle \big)$. Vous devez d'abord construire le `QuantumCircuit` qui prépare cet état.

In [ ]:
qc_ex_3 = QuantumCircuit(5) 
### Construisez votre circuit ici

###
qc_ex_3.draw('mpl')

In [ ]:
average_gain = eval_observable_on_state(gain_operator, qc_ex_3, simulator)
exercise_average_gain(average_gain)

## Outil de visualisation de solution

On vous fourni un outil de visualisation de solution. On doit fournir la forme du graphe comme un `Graph` de `networkx`. On doit également fournir une chaine de bits qui décrit la configuration des équipes. 

**Rappel** : Les chaines de bits se lisent de la droite vers la gauche.

L'équipe 0 apparait comme des cercles blancs, et l'équipe 1, comme des cercles gris.

In [ ]:
graph = nx.Graph()
graph.add_nodes_from([0,1,2,3,4])
graph.add_edges_from([(0, 1), (0, 2), (1, 2), (1, 4), (2, 3), (3, 4)])

x = '01011'  # correspond à l'état `qc_trial` définit précédemment 
print_solution_graph(graph, x)

## Construction du circuit pour QAOA

L'algorithme d'optimisation approximative quantique (QAOA) utilise une forme de circuit quantique particulière. Ce circuit dépend de l'observable de **coût**, qui est la valeur **négative** de l'observable de **gain**. Il contient également des paramètres qui seront ajustés dans une routine d'optimisation.

In [ ]:
cost_operator = - gain_operator
print(cost_operator)

1) Le circuit quantique de QAOA débute en appliquant des portes **Hadamard** sur tous les qubits pour préparer une **superposition** égale de tous les états afin d'exploiter le **parallélisme quantique**.

2) Des séries de portes **RZZ** sont ensuite appliquées en suivant la structure de l'observable de **coût**.

3) L'étape de **mélange** consiste en des rotations **RX** sur chacun des qubits.

Les étapes 2 et 3 peuvent être répétées plusieurs fois grâce au paramètre `reps`.

`Qiskit` fournit la classe `QAOAAnsatz` qui implémente cette structure. 

In [ ]:
qaoa_ansatz_1 = QAOAAnsatz(cost_operator, reps=1)

qaoa_ansatz_1.draw('mpl', scale=0.5)

In [ ]:
qaoa_ansatz_1.decompose(reps=1).draw('mpl', scale=0.5)

In [ ]:
qaoa_ansatz_1.decompose(reps=2).draw('mpl', scale=0.5)

In [ ]:
print(f"Nombre de paramètres dans le circuit: {qaoa_ansatz_1.num_parameters}")
print(qaoa_ansatz_1.parameters)

# Interprétation et visualisation des résultats

L'objectif de l'algoritme QAOA est de préparer un état dont les amplitudes de probabilité les plus élevées correspondent aux solutions recherchées du problème.

On explore cet espace d'états à l'aide des paramètres du circuit QAOA.

Pour une valeur donnée des paramètres, on peut préparer et mesurer l'état correspondant plusieurs fois pour obtenir un petit nombre de solutions candidates qui pourront être vérifiées individuellement.

On présente souvent les résultats d'un calcul quantique sous la forme d'un histogramme.

In [ ]:
params = np.random.random(qaoa_ansatz_1.num_parameters)
print(f"Valeur des paramètres: {params}")

qaoa_ansatz_1_inst = qaoa_ansatz_1.assign_parameters(params)
qaoa_ansatz_1_inst.measure_all()

counts = simulator.run(qaoa_ansatz_1_inst.decompose(reps=2)).result().get_counts()

print(counts)

In [ ]:
plot_histogram(counts, figsize=(8,4))

# Estimation de la valeur moyenne

On peut calculer la valeur moyenne de l'opérateur de coût pour l'état quantique préparé par le circuit QAOA

$  \langle\psi_\text{QAOA}(\boldsymbol{\beta},\boldsymbol{\gamma})|\hat{C}|\psi_\text{QAOA}(\boldsymbol{\beta},\boldsymbol{\gamma})\rangle $.

La classe `Estimator` de Qiskit nous permet d'estimer cette valeur moyenne.

In [ ]:
estimator = Estimator(backend=simulator)

job = estimator.run([(qaoa_ansatz_1.decompose(reps=2), cost_operator, params)])
valeur_moyenne = job.result()[0].data.evs

print(f"Valeur moyenne: {valeur_moyenne}")

## Optimisation des paramètres du circuit quantique de QAOA

On veut maintenant automatiser la recherche des meilleurs paramètres pour le circuit QAOA. Pour cela on doit cependant :

1) Définir la fonction à optimiser.

2) Éxécuter le processus d'optimisation avec un optimiseur classique qui choisira quels paramètres essayer dans le circuit

3) Extraire une solution comme étant l'état le plus probable.

In [44]:
# 1. Définir la fonction de coût classique à optimiser

def fonction_cout(
    params: list[complex], estimator: Estimator, circuit: QuantumCircuit, cost_operator: SparsePauliOp
) -> float:

    job = estimator.run([(circuit, cost_operator, params)])
    cout = job.result()[0].data.evs
    return cout

In [ ]:
params_init = np.zeros(qaoa_ansatz_1.num_parameters)

# 2. Éxécuter le processus d'optimisation à l'aide de la méthode COBYLA de Scipy
res_opt = minimize(
    fonction_cout, params_init, args=(estimator, qaoa_ansatz_1.decompose(reps=2), cost_operator), method="COBYLA"
)

# 3. Extraction des informations suite à l'optimisation
cout_opt = res_opt.fun  # Cout optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

# Affichage des résultats obtenus
print("Cout optimal trouvé :", cout_opt)
print("Paramètres optimaux trouvés :", params_opt)

Lorsqu'on a obtenu les paramètres optimaux - c'est-à-dire ceux qui minimisent la valeur moyenne de l'observable de coût - on prépare l'état quantique correspondant et on obtient les solutions optimales.

In [ ]:
qaoa_ansatz_1_opt = qaoa_ansatz_1.assign_parameters(params_opt)
qaoa_ansatz_1_opt.decompose().draw('mpl', scale=0.5)

In [ ]:
qaoa_ansatz_1_opt.measure_all()

final_counts = simulator.run(qaoa_ansatz_1_opt.decompose(reps=2)).result().get_counts()

plot_histogram(final_counts, figsize=(8,4))

In [ ]:
maximum_prob_state = max(final_counts, key=final_counts.get)
print("État de base avec la plus grande probabilité: ", maximum_prob_state)

In [ ]:
print_solution_graph(graph, maximum_prob_state)

### Remarque

Lorsqu'on utilise un petit nombre de répétitions, l'algorithme QAOA ne retourne pas toujours une bonne solution. Aussi, les bonnes solutions ne se distinguent pas toujours très bien des mauvaises. On va maintenant voir l'effet d'un plus grand nombre de répétitions.

## Effet du nombre de répétitions

Testons QAOA pour le même problème mais utilisons plus de répétitions.

In [ ]:
# Créer le circuit paramétré QAOA avec 8 répétitions
qaoa_ansatz_8 = QAOAAnsatz(cost_operator, reps=8)

# Initialiser la valeur initiale des paramètres à 0
params_init = np.zeros(qaoa_ansatz_8.num_parameters)

# Trouver la valeur des angles qui minimisent l'opérateur de coût
res_opt = minimize(
    fonction_cout, params_init, args=(estimator, qaoa_ansatz_8.decompose(reps=2), cost_operator), method="COBYLA"
)

# Extraire des informations suite à l'optimisation
cout_opt = res_opt.fun  # Coût optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

# Obtenir le circuit qui prépare l'état optimal
qaoa_ansatz_8_opt = qaoa_ansatz_8.assign_parameters(params_opt)
qaoa_ansatz_8_opt.measure_all()

optimal_qc_with_measurements = qaoa_ansatz_8_opt.copy()
optimal_qc_with_measurements.measure_all()
final_counts = simulator.run(qaoa_ansatz_8_opt.decompose(reps=2)).result().get_counts()

# plot the histogram
plot_histogram(final_counts, figsize=(8,4))

## Observations

Vous remarquerez que QAOA ne converge pas sur une bonne solution à tous les coups. Cet algorithme ne s'appelle pas _Algorithme d'Optimisation **Approximative** Quantique_ pour rien!

Cependant, pour des problèmes très complexes, où le nombre de configurations possibles est exponentiellement grand, QAOA pourrait permettre de suggérer des solutions dont la validité est ensuite facile à vérifier.

# Sans optimisation

En se basant sur la théorie du recuit quantique (_quantum annealing_), il est possible de calculer classiquement un ensemble de paramètres qui permet d'obtenir une valeur moyenne qui s'approche d'un résultat optimal. Il n'est cependant pas clair si cet approche continue de fonctionner pour des systèmes de plus grande taille.

In [ ]:
qaoa_ansatz_3 = QAOAAnsatz(cost_operator, reps=3)

betas = [-1., -0.6, -0.2]
gammas = [0.2, 0.6, 1.]

qaoa_ansatz_3_opt = qaoa_ansatz_3.assign_parameters(betas + gammas)
qaoa_ansatz_3_opt.decompose(reps=2).draw(scale=0.5)

In [ ]:
qaoa_ansatz_3_opt.measure_all()
final_counts = simulator.run(qaoa_ansatz_3_opt.decompose(reps=2)).result().get_counts()

plot_histogram(final_counts, figsize=(8,4))

## Formulaires d'évaluation et d'appréciation

Vous pouvez maintenant utiliser et modifier ce notebook pour vous aider à répondre aux questions d'évalution en cliquant [ici](https://forms.office.com/r/xAKdrNQEej). Vous devrez répondre correctement à au moins 2 questions sur 4 pour obtenir votre attestation. Le questionnaire comprend également des questions sur votre appréciation de l'atelier. Une période de 24h vous est allouée pour répondre. Merci!